In [13]:
### Pour utiliser sur google colab

import os

from google.colab import drive
drive.mount('drive')

!mkdir -p drive -v
#!google-drive-ocamlfuse drive

cwd = os.getcwd()

#### Changez le chemin ci-dessous vers votre repertoire dans votre googledrive
##############################################################################

monchemin = 'drive/My Drive/M2-IAAA/TAL/Projet/'

dir_path  = os.path.join(cwd, monchemin)
dirs = os.listdir(dir_path)
os.chdir(dir_path)

###  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [14]:
!ls

Arc-eager.ipynb  UD_French-GSD


In [0]:
class Word:
    def __init__(self):
        self.featDic = {}

    def getFeat(self, featName):
        if(not featName in self.featDic):
            print('WARNING : feat', featName, 'does not exist')
            return None
        else:
            return self.featDic[featName]

    def setFeat(self, featName, featValue):
        self.featDic[featName] = featValue

    def affiche(self, mcd):
        for elt in mcd:
            feat, status = elt
            print(self.getFeat(feat), '\t', end='')
        print('')

        
    @staticmethod
    def fakeWord(mcd):
        w =Word()
        for elt in mcd:
            feat, status = elt
            w.setFeat(feat, 'ROOT')
        return w

    @staticmethod
    def invalidGov():
        return 123456789

    @staticmethod
    def invalidLabel():
        return ''


In [0]:


class WordBuffer:
        def __init__(self, mcd=None):
                self.currentIndex = 0
                self.array = []
                self.length = 0
                self.mcd = mcd
                
        def addWord(self, w):
                self.array.append(w)
                self.length += 1

        def affiche(self, mcd):
                for w in self.array:
                        w.affiche(mcd)

        def getLength(self):
                return self.length
        
        def getCurrentIndex(self):
                return self.currentIndex

        def getWord(self, index):
                return self.array[index]

        def getCurrentWord(self):
                return self.getWord(self.currentIndex)
            
        def nextSentence(self):
                sentence = []
                sentence.append(Word.fakeWord(self.mcd))
                if self.currentIndex == self.length - 1 :
                        return False
                while self.currentIndex < self.length :
                        sentence.append(self.getCurrentWord())
#                        self.getCurrentWord().affiche(self.mcd)
                        if int(self.getCurrentWord().getFeat('EOS')) == 1 :
                                self.currentIndex += 1
                                return sentence
                        else:
                                self.currentIndex += 1
                
        def readFromMcfFile(self, mcfFilename):
                try:
                        mcfFile = open(mcfFilename, encoding='utf-8')
                except IOError:
                        print(mcfFilename, " : ce fichier n'existe pas")
                        exit(1)
                tokens = []
                for ligne in mcfFile:
                        tokens = ligne.split()
                        w = Word()
                        for i in range(0, len(tokens)):
                                if(self.mcd[i][0] == 'GOV'):
                                        w.setFeat(self.mcd[i][0], tokens[i])
                                        w.setFeat('GOVABS', str(self.length + int(tokens[i]))) # compute absolute index of governor
                                else:
                                        w.setFeat(self.mcd[i][0], tokens[i])
                        self.addWord(w)
                mcfFile.close();
                
        def readFromConlluFile(self, conlluFilename):
                conlluFile = open(conlluFilename, encoding='utf-8')
                
                tokens = []
                for ligne in conlluFile:
#                        print(ligne)
                        if ligne[0] == '\n' :
                                self.getWord(self.currentIndex - 1).setFeat('EOS', '1')
                                next
                        elif ligne[0] == '#' :
#                                print("commentaire")
                                next
                        else :
#                                1	Je	il	PRON	_	Number=Sing|Person=1|PronType=Prs	2	nsubj	_	_
#featModel = (('B', 0, 'POS'),('S', 0, 'POS'), ('B', 0, 'GOV'), ('S', 0, 'GOV'), ('B', -1, 'POS'), ('B', 1, 'POS'))
                                tokens = ligne.split()
                                if '-' not in tokens[0]:
                                        w = Word()
                                        for i in range(0, len(tokens)):
                                                if i == 0 :
                                                        w.setFeat('INDEX', tokens[i])
                                                if i == 1 :
                                                        w.setFeat('FORM', tokens[i])
                                                if i == 2 :
                                                        w.setFeat('LEMMA', tokens[i])
                                                if i == 3 :
                                                        w.setFeat('POS', tokens[i])
                                                if i == 4 :
                                                        w.setFeat('X1', tokens[i])
                                                if i == 5 :
                                                        w.setFeat('MORPHO', tokens[i])
                                                if i == 6 :
                                                        w.setFeat('GOV', tokens[i])
                                                if i == 7 :
                                                        w.setFeat('LABEL', tokens[i])
                                                if i == 8 :
                                                        w.setFeat('X2', tokens[i])
                                                if i == 9 :
                                                        w.setFeat('X3', tokens[i])
                                        w.setFeat('EOS', '0')
                                        self.addWord(w)
                conlluFile.close();
                
        def end(self):
                if(self.getCurrentIndex() >= self.getLength()):
                        return True
                else:
                        return False

In [15]:
import sys



mcd =(('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA', 'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'), ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))

wb = WordBuffer(mcd);
wb.readFromConlluFile("UD_French-GSD/fr_gsd-ud-train.conllu");

#wb.affiche(mcd)

def printSentence(sentence, mcd):
    for i in range(0, len(sentence)):
        sentence[i].affiche(mcd)


sentence = wb.nextSentence()
sentNb = 1
while sentence :
    print('sentence', sentNb)
    printSentence(sentence, mcd)
    sentNb += 1
    sentence = wb.nextSentence()

sentence 1
ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	
1 	Les 	le 	DET 	_ 	Definite=Def|Gender=Fem|Number=Plur|PronType=Art 	2 	det 	_ 	_ 	
2 	commotions 	commotion 	NOUN 	_ 	Gender=Fem|Number=Plur 	5 	nsubj 	_ 	_ 	
3 	cérébrales 	cérébral 	ADJ 	_ 	Gender=Fem|Number=Plur 	2 	amod 	_ 	_ 	
4 	sont 	être 	AUX 	_ 	Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin 	5 	aux 	_ 	_ 	
5 	devenu 	devenir 	VERB 	_ 	Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part 	0 	root 	_ 	Correct=devenues 	
6 	si 	si 	ADV 	_ 	_ 	7 	advmod 	_ 	_ 	
7 	courantes 	courant 	ADJ 	_ 	Gender=Fem|Number=Plur 	5 	xcomp 	_ 	_ 	
8 	dans 	dans 	ADP 	_ 	_ 	10 	case 	_ 	_ 	
9 	ce 	ce 	DET 	_ 	Gender=Masc|Number=Sing|PronType=Dem 	10 	det 	_ 	_ 	
10 	sport 	sport 	NOUN 	_ 	Gender=Masc|Number=Sing 	7 	obl:mod 	_ 	_ 	
11 	qu' 	que 	SCONJ 	_ 	_ 	14 	mark 	_ 	SpaceAfter=No 	
12 	on 	on 	PRON 	_ 	Gender=Masc|Number=Sing|Person=3 	14 	nsubj 	_ 	_ 	
13 	les 	le 	PRON 	_ 	Number=Plur|Person=3|PronType=Prs 	14 	ob

KeyboardInterrupt: ignored

In [5]:
from random import randint
import numpy as np

sentence = ["je","m'appel","Baptiste","et","je","suis","a","Marseille"]



def left_arc(stack,buffer,arcs):
  arcs.append((buffer[0],stack[len(stack)-1]))
  stack.pop(len(stack)-1)
  return stack,buffer,arcs
  
def right_arc(stack,buffer,arcs):
  arcs.append((stack[len(stack)-1],buffer[0]))
  word = buffer.pop(0)
  stack.append(word)
  return stack,buffer,arcs
  
def shift(stack,buffer):
  word = buffer.pop(0)
  stack.append(word)
  return stack , buffer
  
def reduce(stack):
  stack.pop(len(stack)-1)
  return stack

def oracle(w1,w2):
  return randint(1,4)

def listing(sentence):
  res = []
  for i in range(len(sentence)):
    res.append(i + 1)
  return res

def print_result(res,sentence):
  sentence = ["root"] + sentence
  for i in range(len(res)):
    w1 , w2 = res[i]
    print(sentence[w1],' -> ', sentence[w2])
    
def parser(sentence):
  buffer = listing(sentence)
  arcs = []
  stack = []
  stack.append(0) ## root
  while ( (len(buffer) != 0) & (len(stack) > 0)):
    gold = oracle(stack[len(stack)-1],buffer[0])
    if ( gold == 1 ):
      stack,buffer,arcs = left_arc(stack,buffer,arcs)
    elif (gold == 2):
      stack,buffer,arcs = right_arc(stack,buffer,arcs)
    elif (gold == 3):
      stack,buffer = shift(stack,buffer)
    elif (gold == 4):
      stack = reduce(stack)
      
  return arcs


res = parser(sentence)
print(res)
print(sentence)
print_result(res,sentence)
    
    

[(3, 2), (4, 5), (6, 4), (6, 3), (6, 0)]
['je', "m'appel", 'Baptiste', 'et', 'je', 'suis', 'a', 'Marseille']
('Baptiste', ' -> ', "m'appel")
('et', ' -> ', 'je')
('suis', ' -> ', 'et')
('suis', ' -> ', 'Baptiste')
('suis', ' -> ', 'root')
